# First-Class Functions

不管别人怎么说或怎么想，我从未觉得Python受到来自函数式语言太多的影响。我非常熟悉命令式语言，虽然我把函数定义为一等对象，但是我并不把Python当作函数式编程语言。  -- Guido van Rossum

I have never consider Python to be heavily influenced by functional languages, no matter what people say or think. I was much familar with imperative languages such as C and Algo68 ans although I had made function first-class objects, I didn't view Python as an functional programming language. -- Guido van Rossum

在Python中，函数是一等对象。编程语言理论家把“一等对象”定义为满足下属条件的程序实体：

* 在运行时创建
* 能赋值给变量或数据结构中的元素
* 能作为参数传给函数
* 能作为函数的返回结果

在Python，整数、字符串和字典都是一等对象。

## Treating a Function  Like an Object

In [1]:
def factorial(n):
    """return n!"""
    return 1 if n < 2 else n * factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'return n!'

In [5]:
type(factorial)

function

## Higher-Order Functions

接受函数为参数，或把函数作为结果返回的是高阶函数（higher-order function）。`map`, `sorted`是higher-order function.

### Modern Replacement for map, filter, and reduce

函数式语言通常会提高`map`, `filter`和`reduce`三个高阶函数。在Python3中，`map`和`filter`还是内置函数，但是由于引入了列表推导和生成器表达式，它们变得没那么重要了。

In [6]:
list(map(factorial, range(6)))

[1, 1, 2, 6, 24, 120]

In [8]:
# 列表推导，list comprehension
[factorial(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [9]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [11]:
# 列表推导，list comprehension

[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

在Python3中，map和filter返回生成器，因此现在的它们的直接替代品是生成器表达式。

在Python2中，reduce是内置函数，但是在Python3中放到了`functools`模块里了。

In [13]:
import functools
import operator

In [15]:
functools.reduce(operator.add, range(100))

4950

In [16]:
sum(range(100))

4950

`sum`和`reduce`的通用思想是把某个操作连续应用到序列的元素上，累计之前的结果，把一系列值归约成一个值。

`all`和`any`也是内置的归约函数。

## Anonymous Functions

lambda关键字在Python表达式创建匿名函数。

Python简单的句法限制了lambda函数的定义体内只能用纯表达式。lambda函数的定义体内不能赋值，也不能使用while和try等Python语句。

lambda句法只是语法糖：与def语句一样，lambda表达式会创建函数对象。

The lambda syntax is just syntactic sugar: a lambda expression creates a function object just like the def statement. This is just one of serveral kinds of callable objects in Python.

## The Seven Flavors of Callable Objects

用户定义的函数(User-defined functions)

    使用def语句或lambda表达式创建。

内置函数(Built-in functions)

    使用C语言(CPython)实现的函数，如len或time.strftime

内置方法(Build-in methods)

    使用C语言实现的方法，如dict.get.

方法(Methods)

    在类中定义体中定义的函数。

类(Class)

    调用类时运行类的`__new__`方法创建一个实例，然后运行`__init__`方法，初始化实例，最后把实例返回给调用方。

类的实例(Class instances)

    如果类定义了`__call__`方法，那么它的实例可以作为函数调用。

生成器函数(Generator Functions)

    使用yield关键字的函数或方法。调用生成器函数返回的是生成器对象。

## User-Defined Callable Types

Not only are Python functions real objects, but arbitrary Python objects may also be made to behave like functions. Implementing a `__call__` method is all it takes.

In [1]:
import random

In [2]:
class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

## Function introspection

In [5]:
class Foo: pass

In [6]:
dir(Foo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

与用户定义的常规类一样，函数使用`__dict__`属性存储赋予它的用户属性。这相当于一种基本形式的注解。

In [7]:
class C: pass

obj = C()

In [8]:
def func(): pass

In [10]:
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

|名称|类型|说明|
|---|---|----|
|`__annotations__` | dict | 参数和返回值的注解|
|`__call__`|method-wrapper|实现()运算符；即可调用对象协议|
|`__closure__`|tuple|函数闭包，即自由变量的绑定（通常是None）|
|`__code__`|code|编译成字节码的函数元数据和函数定义体|
|`__defaults__`|tuple|形式参数的默认值|
|`__get__`|method-wrapper|实现只读描述符协议|
|`__globals__`|dict|函数所在模块的全局变量|
|`__kwdefaults__`|dict|仅限关键字形式参数的默认值|
|`__name__`|str|函数名称|
|`__qualname__`|str|函数限定名称|

In [16]:
func.__kwdefaults__

In [17]:
def func(name='Mac'): pass

In [19]:
func.__defaults__, func.__kwdefaults__

(('Mac',), None)

## From Positional to Keyword-Only Parameters

Keyward-only arguments are a new feature in Python3. To specify keyword-only argument when defining a function, name them after the argument prefixed with `*`. If you don't want to support variable positional argument but still want keyword-only argument, put a `*` by itself in the signature.

In [25]:
def f(a, *, b):
    return a, b

In [27]:
f(1, b=2)

(1, 2)

## Function Annotations

Python3 provides syntax to attatch metadata to the parameters of a function declaration and its return value.

Python3提供了一种语法，用于为函数声明中的参数和返回附加元数据。

In [30]:
def clip(text:str, max_len:'int > 0'=80) -> str: # The annotated function declaration(有注解的函数声明)
    pass

In [31]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

In [32]:
from inspect import signature

In [38]:
sig = signature(clip)
sig._return_annotation

str

## Packages for Functional Programming

Althougth Guido makes it clear that Python does not aim to be a functional programming language, a functional coding style can be used to good extend, thanks to the support of packages like operator and functools.

## The operator Module

在函数式编程中，经常需要把算术运算符当作函数使用。

In [41]:
import operator
import functools

In [40]:
def fact(n):
    return functools.reduce(lambda a, b: a*b, range(1, n+1))

In [42]:
def fact(n):
    return functools.reduce(operator.mul, range(1, n+1))

In [43]:
fact(10)

3628800

operator模块中还有一类函数，能替代从序列取出元素或读取对象属性的lambda表达式，`itemgetter`和`attrgetter`其实会自行构建函数。

Another group of one-trick lambdas that operator replaces are functions to pick item from sequences or read attributes from objects: `itemgetter` and `attrgetter` actually build custom functions to do that.

**`itemgetter`**

A common use of `itemgetter`: sorting a list of tuples by the value of a field.

In [44]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [45]:
for city in sorted(metro_data, key=operator.itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


If you pass multiple index arguments to `itemgetter`, the function it builds will return tuples with the extracted values.

In [46]:
cc_names = operator.itemgetter(1, 0)
for city in metro_data:
    print(cc_names(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


Because `itemgetter` use the `[]` operator, it supports not only sequences but also mappings and any class that implements `__getitem__`.

**`attrgetter`**

A sibling of `itemgetter` is `attrgetter`, which creates functions to extract object attribute by name. If you pass `attrgetter` serveral attribute names as arguments, it also return a tuple of values. In addition, if any argument name contains a `.(dot)`, `attrgetter` navigates througth nested objects retrieve the attributes.

In [47]:
import collections

In [50]:
LatLong = collections.namedtuple('LatLong', 'lat long')
Metropolis = collections.namedtuple('Metropolis', 'name cc pop coord')

In [51]:
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]

In [52]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [53]:
metro_areas[0].coord.lat

35.689722

In [54]:
name_lat = operator.attrgetter('name', 'coord.lat')

In [55]:
for city in sorted(metro_areas, key=operator.attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


**`methodcaller`**

Of the remaing operator functions, `methodcaller` is the last we will cover. It is somewhat similar to `itemgetter` and `attrgetter` in that it create a function on the fly. The function it creates calls a method by name on the object given as argument.

In [58]:
s = 'The time has come'
upper = operator.methodcaller('upper')
upper(s)

'THE TIME HAS COME'

In [59]:
hiphenate = operator.methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

## Freezing Arguments with `functoools.partial`

`functools.partial` is a high-order functions that allows partial application of a function.

In [2]:
import operator
import functools

In [3]:
triple = functools.partial(operator.mul, 3)

In [4]:
triple(7)

21

In [5]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

An impressive `functools` function is `lru_cache`, which does memorization -- a form of automatic optimization that works by storing the results of function calls to avoid expensive recalculations.

## Chapter Summary

The goal of this chapter was to explore the first-class nature of functions in Python.